In [44]:
import os
import json
import time
import requests
import importlib
from planetscopedownload import planetscopedownload
importlib.reload(planetscopedownload)

data_dir = 'data'
api_path = os.path.join(data_dir, 'planetscope', "PlanetScope_API_key.txt")
PLANET_API_KEY = planetscopedownload.load_api_key(api_path)

In [45]:
planetscopedownload.retrieve_imagery(
    sitename='hawaiiwaikiki', 
    start_date='2019-01-01', 
    end_date='2019-01-15', 
    planet_api_key=PLANET_API_KEY,
    data_dir=r'C:\Users\Joel Nic\Documents\college\research\CRC\SDS_performance_analysis\data'
    )

polygon=[[-157.830072, 21.274378], [-157.824826, 21.274378], [-157.824826, 21.277268], [-157.830072, 21.277268], [-157.830072, 21.274378]]
Planets data and orders API authentification successful
4 applicable images
❌ Failed to place order
Status code: 400
Error details: {
  "field": null,
  "general": [
    {
      "message": "Unable to accept order: AOI is invalid: Each linear ring must contain at least 4 positions"
    }
  ]
}


MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?

# Download

In [ ]:
import pathlib


In [3]:
def write_api_key_file(api_key:str, overwrite:bool=False, data_dir:str=os.path.join(os.getcwd(), 'data')):
    sites_dir = os.path.join(data_dir, 'sites')
    if not os.path.exists(sites_dir): os.mkdir(sites_dir)
    
    file_path = os.path.join(sites_dir, 'PlanetScope_API_key.txt')
    if overwrite or not os.path.exists(file_path):
        # if we want to overwrite or if it doesnt exsist we will need to make it
        with open(file_path, "w") as file:
            file.write(api_key)